<a href="https://colab.research.google.com/github/PashaIanko/Kaggle.Spaceship-Titanic/blob/main/5_model_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [10]:
from google.colab import drive

import pandas as pd

import os

## PEP8 standard

In [2]:
# PEP 8 
%%capture 
!pip install pycodestyle 
!pip install --index-url https://test.pypi.org/simple/ nbpep8 
from nbpep8.nbpep8 import pep8

## Importing custom packages

In [3]:
%%capture
def download_files(url_dict):
    for file, url in url_dict.items():
        !wget -O {file} {url} {file}


git_download_path = \
    'https://raw.githubusercontent.com/PashaIanko/Sklearn-Utils/main/'

files = [
    'path_manager.py',
    'sklearn_transformers.py',
    'sklearn_utils.py',
    'model.py'
]

url_dict = {file: git_download_path + file for file in files}
download_files(url_dict)

In [4]:
import importlib
import path_manager
# import sklearn_utils
# import sklearn_transformers
# import model


def reload_modules(modules):
    for module in modules:
        importlib.reload(module)


reload_modules(
    [
        path_manager,
        # sklearn_transformers,
        # sklearn_utils,
        # model
    ]
)

from path_manager import PathManager
# from sklearn_transformers import ColumnDropper
# from sklearn_utils import inf_statistics
# from model import Model

# Set up paths

In [5]:
drive.mount('/content/gdrive', force_remount=True)
gdrive_path = '/content/gdrive/MyDrive/'

PREPROC_TRIAL = 1
MODELS_TRIAL = 1
COMPETITION_PATH = gdrive_path + 'ML/Competitions/6.SpaceshipTitanic/'

manager = PathManager(
    competition_path=COMPETITION_PATH,
    preprocessing_trial=PREPROC_TRIAL,
    models_trial=MODELS_TRIAL
)
manager.setup_paths()

Mounted at /content/gdrive
/content/gdrive/MyDrive/ML/Competitions/6.SpaceshipTitanic/Data/preproc_trial_1 already exists
/content/gdrive/MyDrive/ML/Competitions/6.SpaceshipTitanic/Models/trial_1 already exists


# Download the trained models

In [8]:
models = manager.load_models(
    [
        'RandomForest_gridsearch',
        'LogReg_gridsearch'
    ]
)
models

{'RandomForest_gridsearch': RandomForestClassifier(max_depth=6, max_features='sqrt', n_estimators=75,
                        n_jobs=-1, random_state=42),
 'LogReg_gridsearch': LogisticRegression(C=0.1, l1_ratio=0, penalty='elasticnet', random_state=42,
                    solver='saga')}

# Download the data

In [12]:
os.listdir(manager.data_trial_path)

['train_processed.csv',
 'trainval_processed.csv',
 'test_sample_processed.csv',
 'test_processed.csv']

In [14]:
df_test_sample = pd.read_csv(
    os.path.join(
        manager.data_trial_path,
        'test_sample_processed.csv'
    )
)

In [13]:
df_test_processed = pd.read_csv(
    os.path.join(
        manager.data_trial_path,
        'test_processed.csv'
    )
)

In [15]:
df_trainval_processed = pd.read_csv(
    os.path.join(
        manager.data_trial_path,
        'trainval_processed.csv'
    )
)

# Report test sample performance